In [36]:
import json
import os
import pandas as pd
import glob
import pandasgui as pdg

print(pdg.__version__)
diff_svg_threshold = 1e-4

0.2.13


In [37]:
dataset = ["OpenAPS", "OPENonOH"]
outdir = "/home/reinhold/Daten/OPEN/"
outfilename = f"duplicates_{dataset[0]}_{dataset[1]}.csv"

indir = [f"{outdir}/{d}_Data/csv_per_day/" for d in dataset]
infilename = [f"entries_{d}.csv" for d in dataset]

df = [pd.read_csv(os.path.join(indir[i], infilename[i]), header=0, parse_dates=[1], index_col=0) for i in range(2)]
df[0]["date"] = pd.to_datetime(df[0]["date"],format="%Y-%m-%d")
out = df[0].merge(df[1], left_on="date", right_on="date", how="inner", suffixes=("_OpenAPS", "_OPENonOH"))

print(df[0].columns)
for col in df[0].columns:
    if not "sgv" in col: continue
    out["diff_" + col] = 2*(out[col + "_OpenAPS"] - out[col + "_OPENonOH"])/(out[col + "_OpenAPS"] + out[col + "_OPENonOH"])

df2 = out[(out.diff_sgv_mean*out.diff_sgv_mean < diff_svg_threshold) & (out.diff_sgv_std*out.diff_sgv_std < diff_svg_threshold) & (out.diff_sgv_min*out.diff_sgv_min < diff_svg_threshold) & (out.diff_sgv_max*out.diff_sgv_max < diff_svg_threshold) & (out.diff_sgv_count*out.diff_sgv_count < diff_svg_threshold)]
df3 = df2[["user_id_OpenAPS", "user_id_OPENonOH", "date", "diff_sgv_mean", "diff_sgv_std", "diff_sgv_min", "diff_sgv_max", "diff_sgv_count", "second_id_OPENonOH", "filename_OPENonOH", "filename_OpenAPS"]].sort_values(by=["user_id_OpenAPS", "user_id_OPENonOH", "date"])
#df3 = df3[[]]

# group by statement has to return exactly one line, otherwise the selection criteria on diff_sgv_*-variables need to be tightened.

gui = pdg.show(df3)

df3.to_csv(os.path.join(outdir, outfilename))
print(os.path.join(outdir, outfilename) + " created")


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


Index(['date', 'sgv_mean', 'sgv_std', 'sgv_min', 'sgv_max', 'sgv_count',
       'filename', 'user_id', 'second_id'],
      dtype='object')
/home/reinhold/Daten/OPEN/duplicates_OpenAPS_OPENonOH.csv created


In [38]:
df_duplicates_only = df3.groupby(["diff_sgv_mean", "diff_sgv_std", "diff_sgv_min", "diff_sgv_max", "diff_sgv_count"]).agg(["count"])
if len(df_duplicates_only) > 1: 
    columns = []
    for col in df_duplicates_only.columns:
        columns.append(f"{col[0]}_{col[1]}")
    df_duplicates_only.columns = columns
    df_duplicates_only.fillna(value=0, inplace=True)  #
    df_duplicates_only.reset_index(inplace=True)

    print(df_duplicates_only)
    raise ValueError("df_duplictes_only has to to have only one entry")

In [39]:
# get list of duplicates and the count of days
df3_mod = df3[['user_id_OpenAPS', 'user_id_OPENonOH','date']]
df4 = df3_mod.groupby(['user_id_OpenAPS', 'user_id_OPENonOH']).agg(["count"])

columns = []
for col in df4.columns:
    columns.append(f"{col[0]}_{col[1]}")
df4.columns = columns
df4.fillna(value=0, inplace=True)  #
df4.reset_index(inplace=True)
#print(df4[['user_id_OpenAPS', 'user_id_OPENonOH','date_count']])
print(df4)


   user_id_OpenAPS  user_id_OPENonOH  date_count
0         54280576           8814820          15
1         63725802           1104400         256
2         70454270          72492017         203
3         80625186          77667622         330
4         86323903            749582        1300
5         93606058           9922617         359
6         94200862          12826270          51
